# DENSITY FUNCTIONAL THEORY USING PYSCF 

In this tutorial we will discuss how to perform Density Functional Theory calculations for various molecules using the PySCF library.


## Setup 

### Installing libraries

In [2]:
! pip install pyscf 

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


### Import required modules

In [3]:
from pyscf import gto
from pyscf import dft

## Introduction to DFT

The wave function is a useful mathematical construct for calculating the total energy of atoms and molecules, as well as other features. However, it can be very computationally expensive to compute even for few particle systems. Hence, the DFT is useful for predicting the behaviour of particles in a quantum system. 

### The many body problem 



<img src="~/Desktop/poteq.jpg" width="100" height="40">

This is the potential energy term of the Schodinger's equation of a many particle system. 

<img src="~/Desktop/kineq.jpg" width="100" height="40">

This is the kinetic energy term of the Schodinger's equation of a many particle system.

As we can see from the equations, the summation is over all electrons and ions in the system, which would be of the order of 10^23 particles , resulting in a large number of unknown variables.
Reference: [1]

### Kohn Sham Density Functional Theory


## Building the Molecule 

- The 'gto' command is used to initialise the molecule.
- The three numbers beside the atom determine it's coordinates in the basis system. (most calculations do not use the planar system as it yields inaccurate results)

NOTE: The basis determines the accuracy of the calculations. The number of Kohn Sham equations depends on the basis that has been chosen. If the basis chosen is more localised on the molecule, the number of equations will be less. It is similar to the concept of "data fitting" where the right initial guess equation must be chosen for a better fit. 


The example shown below is for a water molecule 

In [10]:

mol = gto.M(
    atom = '''
    O  0.   0.       0.
    H  0.   2.       0.
    H  0.   0.       2. ''',
    basis = 'ccpvdz')

Exchange Correlation Functional 


Numerical Grids

Either rectangular or radial grids may be used depending on the problem. The examples in this tutorial use rectangular grids. An example below (where the molecule is the water molecule built above) shows the difference between two difference grids being used, just for reference.

In [11]:
method = dft.KS(mol)
print('Default DFT(LDA).  E = %.12f' % method.kernel())
method = dft.KS(mol)
method.grids.radi_method = dft.mura_knowles
print('Changed radial grids for DFT.  E = %.12f' % method.kernel())

converged SCF energy = -75.5217424548572
Default DFT(LDA).  E = -75.521742454857
converged SCF energy = -75.5217423020953
Changed radial grids for DFT.  E = -75.521742302095


Simple Example 1- Hydrogen Fluoride:

This example shows how to calculate the molecular orbital energy using the Kohn Shahm Density Functional theory. Here we are using a pre defined basis and xc functional known as the 

In [18]:
import pyscf
mol = pyscf.M(
    atom = 'H 0 0 0; F 0 0 1.1', 
    basis = '631g',
    symmetry = True,
)
mf = mol.KS()
mf.xc = 'pbe0'
mf.kernel()
mf.analyze()

converged SCF energy = -100.291540824411
Wave-function symmetry = Coov
occupancy for each irrep:     A1  E1x  E1y
                               3    1    1
**** MO energy ****
MO #1 (A1 #1), energy= -24.7492063472877 occ= 2
MO #2 (A1 #2), energy= -1.16511154194312 occ= 2
MO #3 (A1 #3), energy= -0.49772811797453 occ= 2
MO #4 (E1x #1), energy= -0.381914983219932 occ= 2
MO #5 (E1y #1), energy= -0.381914983219933 occ= 2
MO #6 (A1 #4), energy= 0.00758507070440067 occ= 0
MO #7 (A1 #5), energy= 0.731778664296396 occ= 0
MO #8 (E1x #2), energy= 1.21518395118488 occ= 0
MO #9 (E1y #2), energy= 1.21518395118487 occ= 0
MO #10 (A1 #6), energy= 1.3353715570668 occ= 0
MO #11 (A1 #7), energy= 1.66010165376359 occ= 0
 ** Mulliken atomic charges  **
charge of  0H =      0.37107
charge of  1F =     -0.37107
Dipole moment(X, Y, Z, Debye):  0.00000,  0.00000, -2.29483


((array([6.21976590e-01, 6.95178763e-03, 1.99997894e+00, 1.91447253e+00,
         2.54163089e-03, 1.99974706e+00, 1.99974706e+00, 1.45390340e+00,
         2.52935310e-04, 2.52935310e-04, 1.75115125e-04]),
  array([ 0.37107162, -0.37107162])),
 array([ 0.        ,  0.        , -2.29483093]))

In [22]:
import pyscf
mol = pyscf.M(
    atom = 'H 0 0 0; F 0 0 1.0', 
    basis = '631g',
    symmetry = True,
)
mf = mol.KS()
mf.xc = 'pbe0'
mf.kernel()
k=mf.analyze()
print(k)
def gen_moles(mol, disp):
    j=0

    
    coords = mol.atom_coords()
    natoms = len(coords)
    mol_a, mol_s = [],[]
    for j in range(3):
        for i in range(natoms):
            for x in range(3):
                new_coords_a, new_coords_s = coords.copy(), coords.copy()
                new_coords_a[i][x] += disp
                new_coords_s[i][x] -= disp
                atoma = [[mol.atom_symbol(j), coord] for (j, coord) in zip(range(natoms), new_coords_a)]
                atoms = [[mol.atom_symbol(j), coord] for (j, coord) in zip(range(natoms), new_coords_s)]
                print(atoma,)
                mol_a.append(mol.set_geom_(atoma, inplace=False, unit='B'))
                mol_s.append(mol.set_geom_(atoms, inplace=False, unit='B'))
                mf = mol_a[x].KS()
                mf.xc = 'pbe0'
                mf.kernel()
                mf.analyze()
                mf1 = mol_s[x].KS()
                mf1.xc = 'pbe0'
                mf1.kernel()
                mf1.analyze()
        #print(mol_a,mol_s)
        return mol_a, mol_s
    disp=disp+0.1
gen_moles(mol,0.1)

converged SCF energy = -100.305246209068
Wave-function symmetry = Coov
occupancy for each irrep:     A1  E1x  E1y
                               3    1    1
**** MO energy ****
MO #1 (A1 #1), energy= -24.745889354497 occ= 2
MO #2 (A1 #2), energy= -1.18892187229547 occ= 2
MO #3 (A1 #3), energy= -0.516689385426204 occ= 2
MO #4 (E1x #1), energy= -0.388053882357506 occ= 2
MO #5 (E1y #1), energy= -0.388053882357508 occ= 2
MO #6 (A1 #4), energy= 0.0470360786024071 occ= 0
MO #7 (A1 #5), energy= 0.785814247188821 occ= 0
MO #8 (E1x #2), energy= 1.21108844891727 occ= 0
MO #9 (E1y #2), energy= 1.21108844891726 occ= 0
MO #10 (A1 #6), energy= 1.32501446947134 occ= 0
MO #11 (A1 #7), energy= 1.69498413848683 occ= 0
 ** Mulliken atomic charges  **
charge of  0H =      0.38607
charge of  1F =     -0.38607
Dipole moment(X, Y, Z, Debye):  0.00000,  0.00000, -2.22663
((array([6.00657051e-01, 1.32684810e-02, 1.99997770e+00, 1.88313528e+00,
       3.16892672e-03, 1.99968923e+00, 1.99968923e+00, 1.49931768e+

([<pyscf.gto.mole.Mole at 0x10da8e790>,
  <pyscf.gto.mole.Mole at 0x10da8e9d0>])

Result Analysis for example 1

As seen from the output, the 


Citations:
 [1] 